In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

laureates_df = spark.read.option("header", "true") \
                        .option("inferSchema", "true") \
                        .csv("/home/iceberg/data/laureates.csv") \
                        .createOrReplaceTempView("laureates")
prize_laureates_df =  spark.read.option("header", "true") \
                        .option("inferSchema", "true") \
                        .csv("/home/iceberg/data/prize_laureates.csv") \
                        .createOrReplaceTempView("prize_laureates")
prizes_df =  spark.read.option("header", "true") \
                        .option("inferSchema", "true") \
                        .csv("/home/iceberg/data/prizes.csv") \
                        .createOrReplaceTempView("prizes")

24/06/14 05:03:07 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
%%sql
  
WITH top_laureate AS (
    SELECT laureate_id, category, COUNT(*) AS total_awards
    FROM prize_laureates
    GROUP BY laureate_id, category
    ORDER BY total_awards DESC
    LIMIT 1
)
    
SELECT DISTINCT l.id, l.full_name, tl.total_awards, tl.category
FROM top_laureate tl
LEFT JOIN laureates l ON tl.laureate_id = l.id;

-- given the winner that has won the most number of nobel awards is expected to be null. the data extracted is also null

id,full_name,total_awards,category
482,None,3,Peace
